In [6]:
from sklearn import metrics, cluster

import numpy as np
import pandas as pd

In [45]:
data = pd.read_csv('checkins.dat', sep='|', header=0, skipinitialspace=True)
data.dropna(inplace=True)

In [46]:
data

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18
...,...,...,...,...,...,...
1021960,955561,626076.0,20073.0,40.850100,-73.866246,2012-04-13 09:56:48
1021961,955892,674797.0,2297.0,33.748995,-84.387982,2012-04-13 10:56:03
1021962,956377,845102.0,11195.0,42.765366,-71.467566,2012-04-13 12:08:45
1021963,956119,1139114.0,29488.0,42.439479,-83.743830,2012-04-13 11:36:44


In [60]:
import csv
with open('checkins.dat', 'r') as fin:
    with open('checkins.csv', 'w') as fout:
        for line in fin:
            newline = list(map(str.strip, line.split('|')))
            if len(newline) == 6 and newline[3] and newline[4]:
                csv.writer(fout).writerow(newline)

In [61]:
data_train = pd.read_csv('checkins.csv')

In [62]:
data_train

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18
...,...,...,...,...,...,...
396629,955561,626076,20073,40.850100,-73.866246,2012-04-13 09:56:48
396630,955892,674797,2297,33.748995,-84.387982,2012-04-13 10:56:03
396631,956377,845102,11195,42.765366,-71.467566,2012-04-13 12:08:45
396632,956119,1139114,29488,42.439479,-83.743830,2012-04-13 11:36:44


In [157]:
X_train = data_train[['latitude', 'longitude']][0:100000]

In [158]:
clustering = cluster.MeanShift(bandwidth=0.1).fit(X_train)

In [159]:
clustering.labels_

array([ 5,  7, 30, ..., 25, 19,  4], dtype=int64)

In [160]:
centers = clustering.cluster_centers_
print(len(clustering.cluster_centers_))
with open('cordinates.txt', 'w') as f:
    for cor in clustering.cluster_centers_:
        f.write(' '.join(str(j) for j in cor))
        f.write('\n')

3231


In [283]:
labels_unique, labels_counts = np.unique(clustering.labels_, return_counts = True)

In [177]:
labels_unique_new = []
for ind, count in enumerate(labels_counts):
    if count > 15:
        labels_unique_new.append(labels_unique[ind])

In [199]:
centers_new = np.empty((len(labels_unique_new), 2))
for ind, label in enumerate(labels_unique_new):
    centers_new[ind] = centers[label]
        
print(centers_new)

[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ...
 [  41.61853175  -88.44556818]
 [  39.2494686   -77.1821271 ]
 [  38.65877915  -76.8856871 ]]


In [253]:
offices = pd.read_csv('offices.txt', skipinitialspace=True)
offices.info()
offices

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
lat    6 non-null float64
lon    6 non-null float64
dtypes: float64(2)
memory usage: 224.0 bytes


,lat,lon
0,33.751277,-118.188740
1,25.867736,-80.324116
2,51.503016,-0.075479
3,52.378894,4.885084
4,39.366487,117.036146
5,-33.868457,151.205134


In [255]:
def dist(coor1, coor2):
    
    return np.sqrt(np.sum((coor1-coor2)**2))

In [267]:
distance = np.empty((centers_new.shape[0], 1))

for ind, corr in enumerate(centers_new):
    min_dist = dist(coor, offices.loc[0])
    for i in range(1, len(offices)):
        d = dist(corr, offices.loc[i])
        if d < min_dist:
            min_dist = d
    distance[ind] = min_dist

In [278]:
dist_indices_sorted = (np.argsort(distance, axis = 0).reshape(len(distance)))
distances_sorted = distance[dist_indices_sorted]
centers_sorted = centers_new[dist_indices_sorted]

In [280]:
print(distances_sorted[0])
print(centers_sorted[0])
answer = centers_sorted[0]

[0.00783476]
[-33.86063043 151.20477593]


In [281]:
with open('answer.txt', 'w') as f:
    f.write(' '.join(str(i) for i in answer))